Upload documents to Neo4j, make into chunk. Create embeddings of those chunks
Search content from within files using Vector Index made out of chunk embedding

In [ ]:
!pip install langchain langchain-openai langchain_neo4j  -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
def load_properties(path):
    data = {}
    with open(path) as f:
        for line in f:
            if "=" in line:
                k, v = line.split("=", 1)
                data[k.strip()] = v.strip()
    return data

In [ ]:
import os
props = load_properties("openai_key.txt")

os.environ["NEO4J_URI"] = props["NEO4J_URI"]
os.environ["OPENAI_KEY"] = props["OPENAI_KEY"]
os.environ["NEO4J_USERNAME"] = props["NEO4J_USERNAME"]
os.environ["NEO4J_PASSWORD"] = props["NEO4J_PASSWORD"]
#os.environ["NEO4J_DATABASE"] = props["NEO4J_DATABASE"]

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain.schema import Document


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [7]:
folder_path = "TextFiles/"

documents = []

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path) and filename.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        doc = Document(
            page_content=content,
            metadata={
                "source": "local",
                "filename": filename,
                "path": file_path
            }
        )

        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 10 documents


In [8]:
# Service used to create the embeddings
embedding_provider = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_KEY")
)

In [9]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD")
)

In [10]:
new_vector_Index = Neo4jVector.from_documents(
    documents,
    embedding_provider,
    graph=graph,
    index_name="myVectorIndex",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
    create_id_index=True,
)

In [ ]:
#Creating Retrieval Chain

In [15]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [16]:
myllmObject = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    api_key=os.getenv("OPENAI_KEY")
  )

In [17]:
filecontent_retriever = RetrievalQA.from_llm(
    llm=myllmObject,
    retriever=new_vector_Index.as_retriever()
)

In [20]:
response = filecontent_retriever.invoke(
    {"query": "find the name of file having text as file 18"}
)

print(response)

{'query': 'find the name of file having text as file 18', 'result': 'I\'m sorry, but based on the provided context, there is no mention of a file with the text "file 18."'}
